In [ ]:
import shutil
shutil.unpack_archive("/content/cemri.zip", "/content/cemri/")

In [ ]:
import shutil
shutil.copytree("/content/drive/My Drive/cemri/", "/content/cemri/")

'/content/cemri/'

In [ ]:
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img

Using TensorFlow backend.


In [ ]:
#loading images using load_img

#converting images into array

#putting array images into train_imgs

IMG_DIM = (256, 256)
train_files = glob.glob('/content/cemri/trainhis/*')
train_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in train_files]
train_imgs = np.array(train_imgs)

#getting labels from the image name
train_labels = [fn.split('/')[-1].split('.')[0]  for fn  in train_files]

validation_files = glob.glob('/content/cemri/testhis/*')
validation_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in validation_files]
validation_imgs = np.array(validation_imgs)
validation_labels = [fn.split('/')[-1].split('_')[0]  for fn in validation_files]



In [ ]:
print(len(validation_labels),len(train_labels))

print(validation_imgs.shape)

614 2450
(614, 256, 256, 3)


In [ ]:
#normalizing individual channel in images array

train_imgs_scaled = train_imgs.astype('float32')
validation_imgs_scaled  = validation_imgs.astype('float32')
train_imgs_scaled /= 255.
validation_imgs_scaled /= 255.



In [ ]:
batch_size = 64
epochs = 7
input_shape = (256,256, 3)

In [ ]:
#encoding image labels 

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(train_labels)
train_labels_enc = le.transform(train_labels)
#validation_labels_enc = le.transform(validation_labels)

#tranforming labels to catogerical
from keras.utils import to_categorical
train_labels_c = to_categorical(train_labels_enc)
#validation_labels_c = to_categorical(validation_labels_enc)

In [ ]:
#building model

from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.models import Sequential
from keras import optimizers
import tensorflow as tf

model = Sequential()

model.add(Conv2D(16, kernel_size=(3, 3), activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(10000, activation=tf.nn.relu))
model.add(Dense(2450, activation=tf.nn.softmax))

opt = optimizers.Adam() 
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 254, 254, 16)      448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 127, 127, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 125, 125, 64)      9280      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 60, 60, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 256)      

In [ ]:
history = model.fit(x=train_imgs_scaled, y=train_labels_c,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1)

Epoch 1/7


In [ ]:


#model= tf.keras.models.load_model('/content/cnn7_aft_aug.h5')
y = model.predict(validation_imgs)
#print(y)
x = ['glioma', 'meningioma', 'pituitary']
c = []
count,total = 0,0
for i in range(len(y)):
    v = np.argmax(y[i])
    c.append(x[int(v)])
p = [[0,0,0],[0,0,0],[0,0,0]]
for i in range(len(y)):
    total += 1
    if c[i] == validation_labels[i]:count += 1
    #print("predicted:", c[i],", real_names:",validation_labels[i])
    for j in range(len(x)):

        for k in range(len(x)):
            if x[j] == c[i] and x[k] == validation_labels[i]: p[k][j] += 1
print(p)

[[256, 28, 2], [4, 128, 10], [1, 2, 183]]


In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(validation_labels, c)
print(cm)

[[256  28   2]
 [  4 128  10]
 [  1   2 183]]


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(validation_labels, c))

              precision    recall  f1-score   support

      glioma       0.98      0.90      0.94       286
  meningioma       0.81      0.90      0.85       142
   pituitary       0.94      0.98      0.96       186

    accuracy                           0.92       614
   macro avg       0.91      0.93      0.92       614
weighted avg       0.93      0.92      0.92       614



In [ ]:
import shutil
shutil.unpack_archive("/content/cemri.zip", "/content/cemri/")

In [ ]:
import shutil
shutil.copytree("/content/drive/My Drive/cemri/", "/content/cemri/")

'/content/cemri/'

In [ ]:

import os

import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline

import pandas as pd
import numpy as np

from PIL import Image

from skimage.feature import hog
from skimage.color import rgb2grey

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC

from sklearn.metrics import roc_curve, auc

In [ ]:
def create_features(path):
    img = Image.open(path)
    img = np.array(img)
    color_features = img.flatten()
    grey_image = rgb2grey(img)
    hog_features = hog(grey_image, block_norm='L2-Hys', pixels_per_cell=(16, 16))
    flat_features = np.hstack(color_features)
    return flat_features



In [ ]:
train_array,train_labels, test_array, test_labels = [], [], [], []

In [ ]:
from random import shuffle
from glob import glob
files = glob(r"/content/cemri/trainhis/*")
shuffle(files)

In [ ]:
def create_feature_matrix(files):
    features_list = []
    labels = []
    
    for i, file in enumerate(files):
        print(i)
        #if i> 300:break
        image_features = create_features(file)
        if len(image_features) !=262144:continue
        labels.append(file.split('_')[0].split('/')[-1])
        features_list.append(image_features)
        
    feature_matrix = np.array(features_list)
    return (feature_matrix, labels)


In [ ]:
train_matrix, train_labels = create_feature_matrix(files)

In [ ]:
train_matrix.shape,len(train_labels)

((2437, 262144), 2437)

In [ ]:
files2 = glob(r"/content/cemri/testhis/*")
shuffle(files2)

In [ ]:
test_matrix ,test_labels = create_feature_matrix(files2)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
test_matrix.shape, len(test_labels)

((612, 262144), 612)

In [ ]:
x = []
for i in range(feature_matrix.shape[0]):
  if feature_matrix[i].shape[0] != 262144:
    x.append(i)

NameError: ignored

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(train_matrix,train_labels)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
y_pred = rf.predict(test_matrix)

In [ ]:
x

In [ ]:
svm = SVC(kernel='linear', probability=True, random_state=42)

svm.fit(train_matrix,train_labels)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=True, random_state=42, shrinking=True, tol=0.001,
    verbose=False)

In [ ]:
svm.save('svm.h5')

AttributeError: ignored

In [ ]:
#y_pred = svm.predict(test_matrix)

accuracy = accuracy_score(test_labels, y_pred)
print('Model accuracy is: ', accuracy)

Model accuracy is:  0.9395424836601307


In [ ]:
test_labels

In [ ]:
print('Feature matrix shape is: ', test_matrix.shape)

ss = StandardScaler()
bees_stand = ss.fit_transform(test_matrix)

pca = PCA(n_components=500)
bees_pca = ss.fit_transform(bees_stand)
print('PCA matrix shape is: ', bees_pca.shape)

In [ ]:
p = []
for i in test_labels:
  p.append(i.split('/')[-1])
p


content based image retrieval

In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D
from keras.models import Model
from keras.callbacks import TensorBoard
from keras.datasets import mnist
import numpy as np


import glob
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img

Using TensorFlow backend.


In [ ]:
IMG_DIM = (256, 256)
train_files = glob.glob('/content/cemri/trainhis/*')
train_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in train_files]
train_imgs = np.array(train_imgs)

#getting labels from the image name
train_labels = [fn.split('/')[-1].split('_')[0]  for fn  in train_files]

validation_files = glob.glob('/content/cemri/testhis/*')
validation_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in validation_files]
validation_imgs = np.array(validation_imgs)
validation_labels = [fn.split('/')[-1].split('_')[0]  for fn in validation_files]


In [ ]:
x_train = train_imgs[:,:,:,2:]
x_test = validation_imgs[:,:,:,2:]

In [ ]:
x_test.shape

(614, 256, 256, 1)

In [ ]:
x_train.shape

(2450, 256, 256, 1)

In [ ]:

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 256, 256, 1))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), 256, 256, 1))  # adapt this if using `channels_first` image data format

noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)

x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

In [ ]:
def train_model():
    input_img = Input(shape=(256, 256, 1))  # adapt this if using `channels_first` image data format
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same', name='encoder')(x)

    # at this point the representation is (4, 4, 8) i.e. 128-dimensional

    x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

    autoencoder.fit(x_train_noisy, x_train,
                    epochs=20,
                    batch_size=128,
                    shuffle=True,
                    validation_data=(x_test_noisy, x_test),
                    callbacks=[TensorBoard(log_dir='/tmp/tb', histogram_freq=0, write_graph=False)])

    autoencoder.save('autoencoder.h5')

In [ ]:
train_model()


ValueError: ignored